In [333]:
import PyPDF2
import os
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
from collections import Counter
import spacy
nlp = spacy.load("en_core_web_sm")
#import en_core_web_sm
#nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher

In [334]:
cd E:\darvin\r_template

E:\darvin\r_template


In [335]:
#Function to read resumes from the folder one by one
mypath=r'E:\darvin\resume_prase' #enter your path here where you saved the resumes
onlyfiles = [os.path.join(mypath, f) for f in os.listdir(mypath) if os.path.isfile(os.path.join(mypath, f))]


In [336]:
onlyfiles

['E:\\darvin\\resume_prase\\darvin l.pdf',
 'E:\\darvin\\resume_prase\\karthik m.pdf',
 'E:\\darvin\\resume_prase\\raja.pdf']

In [337]:
def pdfextract(file):
    fileReader = PyPDF2.PdfFileReader(open(file,'rb'))
    countpage = fileReader.getNumPages()
    count = 0
    text = []
    while count < countpage:    
        pageObj = fileReader.getPage(count)
        count +=1
        t = pageObj.extractText()
        print (t)
        text.append(t)
    return text

In [338]:
cd E:\darvin\resume_prase

E:\darvin\resume_prase


In [339]:
fileReader=PyPDF2.PdfFileReader(open('darvin l.pdf','rb'))

In [340]:
fileReader

In [341]:
countpage = fileReader.getNumPages()

In [342]:
countpage

1

In [343]:
pageObj= fileReader.getPage(0)

In [344]:
pageObj

{'/Type': '/Page',
 '/Parent': {'/Type': '/Pages', '/Count': 1, '/Kids': [IndirectObject(3, 0)]},
 '/Resources': {'/Font': {'/F1': {'/Type': '/Font',
    '/Subtype': '/TrueType',
    '/Name': '/F1',
    '/BaseFont': '/ABCDEE+Calibri',
    '/Encoding': '/WinAnsiEncoding',
    '/FontDescriptor': {'/Type': '/FontDescriptor',
     '/FontName': '/ABCDEE+Calibri',
     '/Flags': 32,
     '/ItalicAngle': 0,
     '/Ascent': 750,
     '/Descent': -250,
     '/CapHeight': 750,
     '/AvgWidth': 521,
     '/MaxWidth': 1743,
     '/FontWeight': 400,
     '/XHeight': 250,
     '/StemV': 52,
     '/FontBBox': [-503, -250, 1240, 750],
     '/FontFile2': {'/Filter': '/FlateDecode', '/Length1': 562580}},
    '/FirstChar': 32,
    '/LastChar': 121,
    '/Widths': [226,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0,
     0

In [345]:
 t = pageObj.extractText()

In [346]:
t

'D\narvin \nL\n \nM\nsc \nstatistics\n \nS\nkills\n \nD\nata \nanalysis\n \nL\ninear \nregression\n \nS\nvm\n \nS\nql\n \nA\nws\n \nN\normal \ndistribution\n \n \n'

In [347]:
text=[]

In [348]:
text.append(t)

In [349]:
text

['D\narvin \nL\n \nM\nsc \nstatistics\n \nS\nkills\n \nD\nata \nanalysis\n \nL\ninear \nregression\n \nS\nvm\n \nS\nql\n \nA\nws\n \nN\normal \ndistribution\n \n \n']

In [350]:
#function that does phrase matching and builds a candidate profile
def create_profile(file):
    text = pdfextract(file) 
    text = str(text)
    text = text.replace("\\n", "")
    text = text.lower()
    #below is the csv where we have all the keywords, you can customize your own
    keyword_dict = pd.read_csv(r'E:\darvin\r_template\template.csv')
    stats_words = [nlp(text) for text in keyword_dict['Statistics'].dropna(axis = 0)]
    NLP_words = [nlp(text) for text in keyword_dict['NLP'].dropna(axis = 0)]
    ML_words = [nlp(text) for text in keyword_dict['Machine Learning'].dropna(axis = 0)]
    DL_words = [nlp(text) for text in keyword_dict['Deep Learning'].dropna(axis = 0)]
    R_words = [nlp(text) for text in keyword_dict['R Language'].dropna(axis = 0)]
    python_words = [nlp(text) for text in keyword_dict['Python Language'].dropna(axis = 0)]
    Data_Engineering_words = [nlp(text) for text in keyword_dict['Data Engineering'].dropna(axis = 0)]

    matcher = PhraseMatcher(nlp.vocab)
    matcher.add('Stats', None, *stats_words)
    matcher.add('NLP', None, *NLP_words)
    matcher.add('ML', None, *ML_words)
    matcher.add('DL', None, *DL_words)
    matcher.add('R', None, *R_words)
    matcher.add('Python', None, *python_words)
    matcher.add('DE', None, *Data_Engineering_words)
    doc = nlp(text)
    
    d = []  
    matches = matcher(doc)
    for match_id, start, end in matches:
        rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
        span = doc[start : end]  # get the matched slice of the doc
        d.append((rule_id, span.text))      
    keywords = "\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())
    
    ## convertimg string of keywords to dataframe
    df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
    df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
    df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
    df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
    df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))
    
    base = os.path.basename(file)
    filename = os.path.splitext(base)[0]
       
    name = filename.split('_')
    name2 = name[0]
    name2 = name2.lower()
    ## converting str to dataframe
    name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])
    
    dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
    dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)

    return(dataf)

In [351]:
text=pdfextract(onlyfiles[1])

karthick
 
M
sc 
statistics
 
S
kills
 
D
ata 
analysis
 
L
inear 
regression
 
S
vm
 
S
ql
 
A
ws
 
N
ormal 
distribution
 
K
eras
 
 
 



In [352]:
text = str(text)
text = text.replace("\\n", "")
text = text.lower()

In [353]:
text

"['karthick msc statistics skills data analysis linear regression svm sql aws normal distribution keras   ']"

In [354]:
#below is the csv where we have all the keywords, you can customize your own
keyword_dict = pd.read_csv(r'E:\darvin\r_template\template.csv')
stats_words = [nlp(text) for text in keyword_dict['Statistics'].dropna(axis = 0)]
NLP_words = [nlp(text) for text in keyword_dict['NLP'].dropna(axis = 0)]
ML_words = [nlp(text) for text in keyword_dict['Machine Learning'].dropna(axis = 0)]
DL_words = [nlp(text) for text in keyword_dict['Deep Learning'].dropna(axis = 0)]
R_words = [nlp(text) for text in keyword_dict['R Language'].dropna(axis = 0)]
python_words = [nlp(text) for text in keyword_dict['Python Language'].dropna(axis = 0)]
Data_Engineering_words = [nlp(text) for text in keyword_dict['Data Engineering'].dropna(axis = 0)]

In [355]:
matcher = PhraseMatcher(nlp.vocab)
matcher.add('Stats', None, *stats_words)
matcher.add('NLP', None, *NLP_words)
matcher.add('ML', None, *ML_words)
matcher.add('DL', None, *DL_words)
matcher.add('R', None, *R_words)
matcher.add('Python', None, *python_words)
matcher.add('DE', None, *Data_Engineering_words)
doc = nlp(text)
    

In [356]:
doc

['karthick msc statistics skills data analysis linear regression svm sql aws normal distribution keras   ']

In [357]:
matches

[(6594074041342103539, 8, 10),
 (6594074041342103539, 10, 11),
 (7237117249260884669, 11, 12),
 (7237117249260884669, 12, 13),
 (8067978966716690490, 13, 15),
 (17708903769805566974, 15, 16)]

In [358]:
d = []  
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
    span = doc[start : end]  # get the matched slice of the doc
    d.append((rule_id, span.text))      
keywords = "\n".join(f'{i[0]} {i[1]} ({j})' for i,j in Counter(d).items())

In [359]:
d

[('ML', 'linear regression'),
 ('ML', 'svm'),
 ('DE', 'sql'),
 ('DE', 'aws'),
 ('Stats', 'normal distribution'),
 ('DL', 'keras')]

In [360]:
keywords

'ML linear regression (1)\nML svm (1)\nDE sql (1)\nDE aws (1)\nStats normal distribution (1)\nDL keras (1)'

In [361]:
 ## convertimg string of keywords to dataframe
df = pd.read_csv(StringIO(keywords),names = ['Keywords_List'])
df1 = pd.DataFrame(df.Keywords_List.str.split(' ',1).tolist(),columns = ['Subject','Keyword'])
df2 = pd.DataFrame(df1.Keyword.str.split('(',1).tolist(),columns = ['Keyword', 'Count'])
df3 = pd.concat([df1['Subject'],df2['Keyword'], df2['Count']], axis =1) 
df3['Count'] = df3['Count'].apply(lambda x: x.rstrip(")"))

In [362]:
df3

,Subject,Keyword,Count
0,ML,linear regression,1
1,ML,svm,1
2,DE,sql,1
3,DE,aws,1
4,Stats,normal distribution,1
5,DL,keras,1


In [363]:
base = os.path.basename('darvin l.pdf')
filename = os.path.splitext(base)[0]

In [364]:
base

'darvin l.pdf'

In [365]:
filename

'darvin l'

In [366]:
name = filename.split('_')
name2 = name[0]
name2 = name2.lower()

In [367]:
# converting str to dataframe
name3 = pd.read_csv(StringIO(name2),names = ['Candidate Name'])

In [368]:
name3

,Candidate Name
0,darvin l


In [369]:
dataf = pd.concat([name3['Candidate Name'], df3['Subject'], df3['Keyword'], df3['Count']], axis = 1)
dataf['Candidate Name'].fillna(dataf['Candidate Name'].iloc[0], inplace = True)

In [370]:
dataf

,Candidate Name,Subject,Keyword,Count
0,darvin l,ML,linear regression,1
1,darvin l,ML,svm,1
2,darvin l,DE,sql,1
3,darvin l,DE,aws,1
4,darvin l,Stats,normal distribution,1
5,darvin l,DL,keras,1


In [371]:
final_database=pd.DataFrame()
i = 0 
while i < len(onlyfiles):
    file = onlyfiles[i]
    dat = create_profile(file)
    final_database = final_database.append(dat)
    i +=1
    print(final_database)

D
arvin 
L
 
M
sc 
statistics
 
S
kills
 
D
ata 
analysis
 
L
inear 
regression
 
S
vm
 
S
ql
 
A
ws
 
N
ormal 
distribution
 
 

  Candidate Name Subject               Keyword Count
0       darvin l      ML    linear regression      1
1       darvin l      ML                  svm      1
2       darvin l      DE                  sql      1
3       darvin l      DE                  aws      1
4       darvin l   Stats  normal distribution      1
karthick
 
M
sc 
statistics
 
S
kills
 
D
ata 
analysis
 
L
inear 
regression
 
S
vm
 
S
ql
 
A
ws
 
N
ormal 
distribution
 
K
eras
 
 
 

  Candidate Name Subject               Keyword Count
0       darvin l      ML    linear regression      1
1       darvin l      ML                  svm      1
2       darvin l      DE                  sql      1
3       darvin l      DE                  aws      1
4       darvin l   Stats  normal distribution      1
0      karthik m      ML    linear regression      1
1      karthik m      ML                  

In [372]:
final_database2 = final_database['Keyword'].groupby([final_database['Candidate Name'], final_database['Subject']]).count().unstack()
final_database2.reset_index(inplace = True)
final_database2.fillna(0,inplace=True)

In [373]:
final_database2

Subject,Candidate Name,DE,DL,ML,Stats
0,darvin l,2.0,0.0,2.0,1.0
1,karthik m,2.0,1.0,2.0,1.0
2,raja,2.0,1.0,1.0,2.0
